# Imports

In [1]:
from ortools.sat.python import cp_model
from calendar import monthrange

from scheduling_funtions import *
from itertools import permutations

# Creating Data

In [2]:
staff_list = ['Olivia', 'Emma', 'Ava', 'Charlotte', 'Sophia', 'Amelia', 'Isabella', 
        'Mia', 'Evelyn', 'Harper', 'Camila', 'Gianna','Abigail', 'Luna', 'Ella',
        'Elizabeth', 'Sofia', 'Emily', 'Avery', 'Mila']

ft_only = [0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
midnight_only = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
first_six_month_only = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1]

staff = range(len(staff_list))
midnight_staff = list(filter(lambda x: midnight_only[x] == 1, staff))
six_month_new_staff = list(filter(lambda x: first_six_month_only[x] == 1, staff))
ft_staff = list(filter(lambda x: ft_only[x] == 1, staff))

def not_staff(staff_list):
    return staff if staff_list == [] else list(set(staff) - set(staff_list))

In [3]:
shift_list = ['0700 - 1500',
          '0730 - 1530 (FT)',
          '0930 - 1730',
          '1200 - 2000',
          '1400 - 2200',
          '1530 - 2330 (FT)',
          '1600 - 2400',
          '1800 - 0200',
          '2000 - 0400',
          '2200 - 0400',
          '2359 - 0700']

shifts = range(len(shift_list))
midnight_shifts = [10]
late_shifts = shifts[7:10]
day_shifts = shifts[:3]
afternoon_shifts = shifts[3:7]
ft_shifts = [2, 5]

def not_shifts(shift_list):
    return shifts if shift_list == [] else list(set(shifts) - set(shift_list))

In [4]:
planning_period = monthrange(2021, 5)
days = range(planning_period[1])

# Creating the model

In [5]:
model = cp_model.CpModel()

In [6]:
#decision variables
#staff 'm' works shift 's' on day 'd'
works = {(m, d, s) : \
    model.NewBoolVar('works_s%id%is%i' % (m, d, s)) \
    for m in staff \
    for d in days \
    for s in shifts}

In [7]:
#intermediate variables
#staff 'm' works on day 'd'

#This enforces the constraint
#No two shifts same day
days_assigned = {(m, d) : \
    model.NewBoolVar('days_works_s%id%i' % (m, d)) \
    for m in staff \
    for d in days}

for m in staff:
  for d in days:
    model.Add(days_assigned[(m,d)] == sum(works[(m,d,s)] for s in shifts))

In [8]:
#intermediate variables
#staff 'm' works on day 'd' on midnight shift 's'
afternoon_shifts_assigned = {(m, d) : \
    model.NewBoolVar('afternoon_s%id%i' % (m, d)) \
    for m in staff \
    for d in days}

for m in staff:
  for d in days:
    model.Add(afternoon_shifts_assigned[(m,d)] == sum(works[(m,d,s)] for s in afternoon_shifts))

In [9]:
#intermediate variables
#staff 'm' works on day 'd' on midnight shift 's'
midnight_shifts_assigned = {(m, d) : \
    model.NewBoolVar('midnight_s%id%i' % (m, d)) \
    for m in staff \
    for d in days}

for m in staff:
  for d in days:
    model.Add(midnight_shifts_assigned[(m,d)] == sum(works[(m,d,s)] for s in midnight_shifts))

In [10]:
#intermediate variables
#staff 'm' works on day 'd' on ft shift 's'
ft_shifts_assigned = {(m, d) : \
    model.NewBoolVar('ft_s%id%i' % (m, d)) \
    for m in staff \
    for d in days}

for m in staff:
  for d in days:
    model.Add(ft_shifts_assigned[(m,d)] == sum(works[(m,d,s)] for s in ft_shifts))

In [11]:
#intermediate variables
#staff 'm' works on day 'd' on late shift 's'
late_shifts_assigned = {(m, d) : \
    model.NewBoolVar('late_s%id%i' % (m, d)) \
    for m in staff \
    for d in days}

for m in staff:
  for d in days:
    model.Add(late_shifts_assigned[(m,d)] == sum(works[(m,d,s)] for s in late_shifts))

In [12]:
#intermediate variables
#staff 'm' works on day 'd' on late shift 's'
day_shifts_assigned = {(m, d) : \
    model.NewBoolVar('day_s%id%i' % (m, d)) \
    for m in staff \
    for d in days}

for m in staff:
  for d in days:
    model.Add(day_shifts_assigned[(m,d)] == sum(works[(m,d,s)] for s in day_shifts))

In [13]:
 obj_int_vars = []
 obj_int_coeffs = []
 obj_bool_vars = []
 obj_bool_coeffs = []

# Constraints

In [14]:
def all_shifts_taken():
  # All shifts should be taken by doctors
  # No two doctors in the same shift on the same day
  for d in days:
      for s in shifts:
        constraint = [works[(m,d,s)] \
                      for m in staff]
        model.Add(sum(constraint) == 1)

def max_days_worked():
    # Maximum work 7 consecutive days
    # SOFT Work maximum of 5 days in a row
    for m in staff:
      variables, coeffs = add_soft_sequence_max_constraint(
          model = model,
          prefix = "max_days_in_a_row",
          shifts = not_list([days_assigned[m, d] for d in days]),
          soft_max = 5,
          hard_max = 7,
          max_cost = 4,
      )
      obj_bool_vars.extend(variables)
      obj_bool_coeffs.extend(coeffs)

def days_off_after_midnight():
  # 2 days off after last midnight (except on call shift).
  # SOFT 3 days off after midnight. 4 even better
  for m in not_staff(midnight_staff): 
    variables, coeffs = add_soft_sequence_min_constraint(
        model = model,
        prefix = "days_off_after_midnight_shift",
        shifts = [days_assigned[(m,d)] for d in days],
        hard_min = 2,
        soft_min = 4,
        min_cost = 2,
        prior = Prior([midnight_shifts_assigned[(m, d)] for d in days], [1], True)
    )
    obj_bool_vars.extend(variables)
    obj_bool_coeffs.extend(coeffs)
    

def midnight_physicians():
  # Certain physicians work only Midnight shifts
  for m in midnight_staff:
      constraint = [works[(m,d,s)] \
                  for d in days \
                  for s in not_shifts(midnight_shifts)]

      model.Add(sum(constraint) == 0)

def no_midnights_within_six_months():
  # Physicians in their first 6 months cant work midnights
  for m in six_month_new_staff:
      constraint = [works[(m,d,s)] \
                  for d in days \
                  for s in midnight_shifts]

      model.Add(sum(constraint) == 0)

def max_midnights_in_a_row():
  # Maximum 2 midnights in a row (except for several physicians who only work midnights)
  # SOFT Max 1 midnight in a row
  for m in not_staff(midnight_staff):
      variables, coeffs = add_soft_sequence_max_constraint(
          model = model,
          prefix = "max_midnight_shifts_in_a_row",
          shifts = not_list([midnight_shifts_assigned[m, d] for d in days]),
          soft_max = 1,
          hard_max = 2,
          max_cost = 3,
      )
      obj_bool_vars.extend(variables)
      obj_bool_coeffs.extend(coeffs)

def on_call_shift_day_after():
  # On call shift - day after rules
  return

def ft_physicians():
  # Certain physicians work only FT shift (0730,1530 shift)
  for m in ft_staff:
    constraint = [works[(m,d,s)] \
                  for d in days \
                  for s in not_shifts(ft_shifts)]

    model.Add(sum(constraint)== 0)

def no_late_shift_before_time_off():
  # No 2000, 2200, or midnight shift prior to day requested off
  return

def no_early_shifts_before_on_call():
  # Physicians can work the 0930 shifts or earlier prior to working on call. They can work starting no earlier than 11 the day after on call.
  return

def days_off_after_consecutive_shifts():
  # 2 days off after 3 to 7 days of work in a row
  for m in staff:
      variables, coeffs = add_soft_sequence_min_constraint(
          model = model,
          prefix = "days_off_after_consecutive_shifts",
          shifts = [days_assigned[m, d] for d in days],
          hard_min = 2,
          soft_min = 2,
          min_cost = 2,
          prior = Prior([days_assigned[m, d] for d in days], [True, True, True]))
      obj_bool_vars.extend(variables)
      obj_bool_coeffs.extend(coeffs)

# Soft Constraints

In [15]:
def transitions_constraints():
    # General principle avoid shift times changing too much day to day
    # Shifts should have same start time to 2.5 hours later compared to previous shift (the 2 hours later can be relaxed to 3,4 perhaps)
    # No shifts that start more than 1.5 hours earlier than the shift on the previous day
    penalized_transitions = []

    for shift in list(permutations(shift_list, 2)):
        t1 = float(shift[0][0:2] + '.' + shift[0][2:4])
        t2 = float(shift[1][0:2] + '.' + shift[1][2:4])
        if t2 - t1 > 2.5:
            penalized_transitions.append(((shift_list.index(shift[0]), shift_list.index(shift[1])), 1))
        elif t2 - t1 < -1.5:
            penalized_transitions.append(((shift_list.index(shift[0]), shift_list.index(shift[1])), 3))

    for (previous_shift, next_shift), cost in penalized_transitions:
        for m in staff:
            for d in days[:-1]:
                transition = [
                    works[m, d, previous_shift].Not(), works[m, d + 1, next_shift].Not()
                ]
                if cost != 0:
                    trans_var = model.NewBoolVar(
                        'transition (employee=%i, day=%i)' % (m, d))
                    transition.append(trans_var)
                    model.AddBoolOr(transition)
                    obj_bool_vars.append(trans_var)
                    obj_bool_coeffs.append(cost)

def days_off_between_late_and_day_shifts():
    # 3 days off when transitioning from late shift to day shift 
    for m in staff:
        add_soft_sequence_min_constraint(
            model = model,
            prefix = "days_off_between_late_and_day",
            shifts = [days_assigned[(m, d)] for d in days],
            hard_min = 0,
            soft_min = 3,
            min_cost = 2,
            prior = Prior([late_shifts_assigned[(m, d)] for d in days], [True]),
            post = Post([day_shifts_assigned[(m, d)] for d in days], [True]),
        )

def days_off_between_late_and_afternoon_shifts():
    # 2 days off when transitioning from late shift to afternoon shift (although this transition should be avoided) 
    for m in staff:
        add_soft_sequence_min_constraint(
            model = model,
            prefix = "days_off_between_late_and_day",
            shifts = [days_assigned[(m, d)] for d in days],
            hard_min = 0,
            soft_min = 2,
            min_cost = 2,
            prior = Prior([late_shifts_assigned[(m, d)] for d in days], [True]),
            post = Post([afternoon_shifts_assigned[(m, d)] for d in days], [True]),
        )

def late_shifts_in_a_row():
    # 3 late shifts in a row maximum - late shifts are 1800, 2000, 2200. The ability to set what a late shift is would be great. LIkely better to set them  
    # as 2000 and 2200. Midnight shift should also be included.								
    for m in staff:
        variables, coeffs = add_soft_sequence_max_constraint(
            model = model,
            prefix = "max_late_shifts_in_a_row",
            shifts = not_list([late_shifts_assigned[m, d] for d in days]),
            soft_max = 3,
            hard_max = 7,
            max_cost = 3,
        )
        obj_bool_vars.extend(variables)
        obj_bool_coeffs.extend(coeffs)

def late_shifts_in_weeks():
    #  5 late shifts in two weeks maximum
    for m in staff:
        for w in range(days[-1] // 7):
            variables, coeffs = add_soft_sum_constraint(
                model = model,
                prefix="max_late_shifts_in_two_weeks",
                shifts = [late_shifts_assigned[m, d + w * 7] for d in range(7)],
                hard_min = 0,
                soft_min = 0,
                min_cost = 0,
                soft_max = 5,
                hard_max = 14,
                max_cost = 3,
                )
            obj_int_vars.extend(variables)
            obj_int_coeffs.extend(coeffs)

def avoid_consecutive_ft_shifts():
    # Avoid FT shifts (0730,1530) on consecutive days
    for m in not_staff(ft_staff):
        variables, coeffs = add_soft_sequence_max_constraint(
            model = model,
            prefix="avoid_consecutive_shifts",
            shifts = not_list([ft_shifts_assigned[(m, d)] for d in days]),
            soft_max=1,
            hard_max=2,
            max_cost=1,)
        obj_int_vars.extend(variables)
        obj_int_coeffs.extend(coeffs)

# Solving the Model

In [16]:
all_shifts_taken()
max_days_worked()
days_off_after_midnight()
midnight_physicians()
no_midnights_within_six_months()
max_midnights_in_a_row()
on_call_shift_day_after()
ft_physicians()
no_late_shift_before_time_off()
no_early_shifts_before_on_call()
days_off_after_consecutive_shifts()

transitions_constraints()
days_off_between_late_and_day_shifts()
days_off_between_late_and_afternoon_shifts()
late_shifts_in_a_row()
late_shifts_in_weeks()
avoid_consecutive_ft_shifts()


In [17]:
import time

start = time.time()

model.Minimize(
        sum(obj_bool_vars[i] * obj_bool_coeffs[i]
            for i in range(len(obj_bool_vars))) +
            sum(obj_int_vars[i] * obj_int_coeffs[i]
            for i in range(len(obj_int_vars))))

solver = cp_model.CpSolver()
solver.parameters.max_time_in_seconds = 600.0
status = solver.Solve(model)

print(solver.StatusName(status))

for d in days:
    print('Day', d)
    for s in shifts:
        for m in staff:
            if solver.Value(works[(m, d, s)]) == 1:
              print(staff_list[m], 'works shift', shift_list[s])
    print()

end = time.time()
print(f"time elapsed: {end - start}")

FEASIBLE
Day 0
Abigail works shift 0700 - 1500
Isabella works shift 0730 - 1530 (FT)
Sofia works shift 0930 - 1730
Mila works shift 1200 - 2000
Gianna works shift 1400 - 2200
Luna works shift 1530 - 2330 (FT)
Emma works shift 1600 - 2400
Camila works shift 1800 - 0200
Amelia works shift 2000 - 0400
Evelyn works shift 2200 - 0400
Mia works shift 2359 - 0700

Day 1
Harper works shift 0700 - 1500
Olivia works shift 0730 - 1530 (FT)
Sophia works shift 0930 - 1730
Ella works shift 1200 - 2000
Mila works shift 1400 - 2200
Charlotte works shift 1530 - 2330 (FT)
Emily works shift 1600 - 2400
Emma works shift 1800 - 0200
Amelia works shift 2000 - 0400
Ava works shift 2200 - 0400
Elizabeth works shift 2359 - 0700

Day 2
Avery works shift 0700 - 1500
Olivia works shift 0730 - 1530 (FT)
Sophia works shift 0930 - 1730
Gianna works shift 1200 - 2000
Camila works shift 1400 - 2200
Charlotte works shift 1530 - 2330 (FT)
Luna works shift 1600 - 2400
Isabella works shift 1800 - 0200
Evelyn works shift 2

# Tests

In [18]:
days_worked_results = {}
midnights_worked_results = {}
lates_worked_results = {}
day_shifts_worked_results = {}
shift_results = {}
ft_worked_results = {}
for m in staff:
    days_worked_results[m] = [solver.Value(days_assigned[(m, d)]) for d in days]
    midnights_worked_results[m] = [solver.Value(midnight_shifts_assigned[(m, d)]) for d in days]
    lates_worked_results[m] = [solver.Value(late_shifts_assigned[(m, d)]) for d in days]
    day_shifts_worked_results[m] = [solver.Value(day_shifts_assigned[(m, d)]) for d in days]
    ft_worked_results[m] = [solver.Value(ft_shifts_assigned[(m, d)]) for d in days]
    for d in days:
        shift_results[(m, d)] = [solver.Value(works[(m, d, s)]) for s in shifts]

## Hard Constraints

In [19]:
# No two doctors in the same shift on the same day
# All shifts should be taken by doctors
for d in days:
    num_staff_per_shift = zip(*[shift_results[(m, d)] for m in staff])
    assert(list(sum(column) for column in num_staff_per_shift) == [1] * len(shifts))
  
#No two shifts same day
for key, result in shift_results.items():
    assert(sum(result) <= 1)

# Maximum work 7 consecutive days
for key, result in days_worked_results.items():
    assert(not detect_pattern(result, "11111111"))

# # 2 days off after last midnight (except on call shift).
for key in midnights_worked_results.keys() and days_worked_results.keys():
    if midnight_only[key]:
         continue
    for idx in days[:-3]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            assert(sum(days_worked_results[key][idx+1:idx+3]) == 0)

# Certain staff work midnights only
for key, result in shift_results.items():
    if (midnight_only[key[0]] == 1):
        assert(result == [0,0,0,0,0,0,0,0,0,0,1] or result == [0] * len(shifts))

# No midnights for staff in their first 6 months (need way to indicate when physician is in first 6 months of practice)
for key, result in midnights_worked_results.items():
    if (first_six_month_only[key]):
        assert(sum(result) == 0)

# Maximum 2 midnights in a row (except for several physicians who only work midnights)
for key, result in midnights_worked_results.items():
    if midnight_only[key]:
         continue
    if (midnight_only[key] == 0):
        assert(not detect_pattern(result, "111"))

# On call shift - day after rules

# Certain physicians work only FT shift (0730,1530 shift)
for key, result in shift_results.items():
    if (staff_list[key[0]] == 1):
        assert(result == [0,1,0,0,0,0,0,0,0,0,0] or result == [0,0,0,0,0,1,0,0,0,0,0] or result == [0] * len(shifts))

# No 2000, 2200, or midnight shift prior to day requested off

# Physicians can work the 0930 shifts or earlier prior to working on call. They can work starting no earlier than 11 the day after on call.

# 2 days off after 3 to 7 days of work in a row
for key, result in days_worked_results.items():
    assert(not detect_pattern(result, "11101"))

print("ALL HARD CONSTRAINTS SATISFIED")

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
ALL HARD CONSTRAINTS SATISFIED


## Soft Constraints

In [20]:
# General principle avoid shift times changing too much day to day
# Shifts should have same start time to 2.5 hours later compared to previous shift (the 2 hours later can be relaxed to 3,4 perhaps)
# No shifts that start more than 1.5 hours earlier than the shift on the previous day
penalized_transitions = []

for shift in list(permutations(shift_list, 2)):
    t1 = float(shift[0][0:2] + '.' + shift[0][2:4])
    t2 = float(shift[1][0:2] + '.' + shift[1][2:4])
    if t2 - t1 > 2.5:
        penalized_transitions.append(((shift_list.index(shift[0]), shift_list.index(shift[1])), 1))
    elif t2 - t1 < -1.5:
        penalized_transitions.append(((shift_list.index(shift[0]), shift_list.index(shift[1])), 3))


transitions = {elem[0]: elem[1] for elem in penalized_transitions}
jumps_forward_25 = 0
jumps_backwards_15 = 0
previous_shift = -1
for key, result in shift_results.items():
    if previous_shift == -1:
        try:
            previous_shift = result.index(1)
        except:
            previous_shift = -1
    else:
        try:
            ans = transitions[(previous_shift, result.index(1))]
            if ans == 1:
                jumps_forward_25 += 1
            elif ans == 3:
                jumps_backwards_15 += 1
            previous_shift = result.index(1)
        except:
            previous_shift = -1

    if key[1] == len(days) - 1:
        previous_shift = -1 

print(f"Number of times shifts jump forwards 2.5 hours: {jumps_forward_25}")
print(f"Number of times shifts jump backwards 1.5 hours: {jumps_forward_25}")
print()

Number of times shifts jump forwards 2.5 hours: 5
Number of times shifts jump backwards 1.5 hours: 5



In [21]:
# 3 days off after midnight. 4 even better
two_days_after_midnight = 0
three_days_after_midnight = 0
four_days_after_midnight = 0
five_days_after_midnight = 0
for key in midnights_worked_results.keys() and days_worked_results.keys():
    for idx in days[:-4]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+3]) == 0 and sum(days_worked_results[key][idx+1:idx+4]) == 1:
                two_days_after_midnight += 1
    for idx in days[:-5]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+4]) == 0 and sum(days_worked_results[key][idx+1:idx+5]) == 1:
                three_days_after_midnight += 1
    for idx in days[:-6]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+5]) == 0 and sum(days_worked_results[key][idx+1:idx+6]) == 1:
                four_days_after_midnight += 1
    for idx in days[:-6]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+6]) == 0:
                five_days_after_midnight += 1
print(f"Number of times an employee gets 2 days off after midnights: {two_days_after_midnight}")
print(f"Number of times an employee gets 3 days off after midnights: {three_days_after_midnight}")
print(f"Number of times an employee gets 4 days off after midnights: {four_days_after_midnight}")
print(f"Number of times an employee gets 5 days off after midnights: {five_days_after_midnight}")
print()

Number of times an employee gets 2 days off after midnights: 2
Number of times an employee gets 3 days off after midnights: 1
Number of times an employee gets 4 days off after midnights: 8
Number of times an employee gets 5 days off after midnights: 1



In [22]:
#Max 1 midnight in a row
two_midnights_in_a_row = 0
for key, result in midnights_worked_results.items():
    two_midnights_in_a_row += detect_pattern_soft(result, '11')
print(f"Number of times an employee works 2 midnights in a row: {two_days_after_midnight}")
print()

Number of times an employee works 2 midnights in a row: 2



In [23]:
# 3 days off when transitioning from late shift to day shift 
one_days_off_from_late_to_day = 0
two_days_off_from_late_to_day = 0
three_days_off_from_late_to_day = 0
for key in day_shifts_worked_results.keys() and lates_worked_results.keys():
    for idx, elem in enumerate(lates_worked_results[key][:-2]):
        if elem == 1 and day_shifts_worked_results[key][idx + 2]:
            if sum(days_worked_results[key][idx+1:idx+2]) == 0:
                one_days_off_from_late_to_day += 1
    for idx, elem in enumerate(lates_worked_results[key][:-3]):
        if elem == 1 and day_shifts_worked_results[key][idx + 3]:
            if sum(days_worked_results[key][idx+1:idx+3]) == 0:
                two_days_off_from_late_to_day += 1
    for idx, elem in enumerate(lates_worked_results[key][:-4]):
        if elem == 1 and day_shifts_worked_results[key][idx + 4]:
            if sum(days_worked_results[key][idx+1:idx+4]) == 0:
                three_days_off_from_late_to_day += 1
print(f"Number of times there is a day shift within 1 days of a late shift: {one_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 2 days of a late shift: {two_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 3 days of a late shift: {three_days_off_from_late_to_day}")
print()


Number of times there is a day shift within 1 days of a late shift: 8
Number of times there is a day shift within 2 days of a late shift: 2
Number of times there is a day shift within 3 days of a late shift: 1



In [24]:
# 2 days off when transitioning from late shift to afternoon shift (although this transition should be avoided) 
one_days_off_from_late_to_afternoon = 0
two_days_off_from_late_to_afternoon = 0
for key in day_shifts_worked_results.keys() and lates_worked_results.keys():
    for idx, elem in enumerate(lates_worked_results[key][:-2]):
        if elem == 1 and day_shifts_worked_results[key][idx + 2]:
            if sum(days_worked_results[key][idx+1:idx+2]) == 0:
                one_days_off_from_late_to_afternoon += 1
    for idx, elem in enumerate(lates_worked_results[key][:-3]):
        if elem == 1 and day_shifts_worked_results[key][idx + 3]:
            if sum(days_worked_results[key][idx+1:idx+3]) == 0:
                two_days_off_from_late_to_afternoon += 1
print(f"Number of times there is a afternoon shift within 1 days of a late shift: {one_days_off_from_late_to_afternoon}")
print(f"Number of times there is a afternoon shift within 2 days of a late shift: {two_days_off_from_late_to_afternoon}")
print()

Number of times there is a afternoon shift within 1 days of a late shift: 8
Number of times there is a afternoon shift within 2 days of a late shift: 2



In [25]:
# 3 late shifts in a row maximum - late shifts are 1800, 2000, 2200. The ability to set what a late shift is would be great
three_days_in_a_row_late = 0
four_days_in_a_row_late = 0
five_days_in_a_row_late = 0
for key, result in lates_worked_results.items():
    three_days_in_a_row_late += detect_pattern_soft(result, '111')
    four_days_in_a_row_late += detect_pattern_soft(result, '1111')
    five_days_in_a_row_late += detect_pattern_soft(result, '11111')
print(f"Number of times an employee works 3 late days in a row: {three_days_in_a_row_late}")
print(f"Number of times an employee works 4 late days in a row: {four_days_in_a_row_late}")
print(f"Number of times an employee works 5 late days in a row: {five_days_in_a_row_late}")
print()

Number of times an employee works 3 late days in a row: 3
Number of times an employee works 4 late days in a row: 0
Number of times an employee works 5 late days in a row: 0



In [26]:
# 5 late shifts in two weeks maximum


In [27]:
# Work maximum of 5 days in a row
#Max 1 midnight in a row
five_days_in_a_row = 0
six_days_in_a_row = 0
seven_days_in_a_row = 0
for key, result in days_worked_results.items():
    five_days_in_a_row += detect_pattern_soft(result, '11111')
    six_days_in_a_row += detect_pattern_soft(result, '111111')
    seven_days_in_a_row += detect_pattern_soft(result, '1111111')
print(f"Number of times an employee works 5 days in a row: {five_days_in_a_row}")
print(f"Number of times an employee works 6 days in a row: {six_days_in_a_row}")
print(f"Number of times an employee works 7 days in a row: {seven_days_in_a_row}")
print()

Number of times an employee works 5 days in a row: 0
Number of times an employee works 6 days in a row: 0
Number of times an employee works 7 days in a row: 0



In [28]:
# Avoid FT shifts (0730,1530) on consecutive days
two_ft_days_in_a_row = 0
for key, result in ft_worked_results.items():
    if ft_only[key]:
        continue
    two_ft_days_in_a_row += detect_pattern_soft(result, '11')
print(f"Number of times an employee works 2 ft shifts in a row: {two_ft_days_in_a_row}")
print()

Number of times an employee works 2 ft shifts in a row: 0



In [29]:
print("SUMMARY")
print()
print(f"Number of times shifts jump forwards 2.5 hours: {jumps_forward_25}")
print(f"Number of times shifts jump backwards 1.5 hours: {jumps_forward_25}")
print()
print(f"Number of times an employee gets 2 days off after midnights: {two_days_after_midnight}")
print(f"Number of times an employee gets 3 days off after midnights: {three_days_after_midnight}")
print(f"Number of times an employee gets 4 days off after midnights: {four_days_after_midnight}")
print(f"Number of times an employee gets 5 days off after midnights: {five_days_after_midnight}")
print()
print(f"Number of times an employee works 2 midnights in a row: {two_days_after_midnight}")
print()
print(f"Number of times there is a day shift within 1 days of a late shift: {one_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 2 days of a late shift: {two_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 3 days of a late shift: {three_days_off_from_late_to_day}")
print()
print(f"Number of times there is a afternoon shift within 1 days of a late shift: {one_days_off_from_late_to_afternoon}")
print(f"Number of times there is a afternoon shift within 2 days of a late shift: {two_days_off_from_late_to_afternoon}")
print()
print(f"Number of times an employee works 3 late days in a row: {three_days_in_a_row_late}")
print(f"Number of times an employee works 4 late days in a row: {four_days_in_a_row_late}")
print(f"Number of times an employee works 5 late days in a row: {five_days_in_a_row_late}")
print()
print(f"Number of times an employee works 5 days in a row: {five_days_in_a_row}")
print(f"Number of times an employee works 6 days in a row: {six_days_in_a_row}")
print(f"Number of times an employee works 7 days in a row: {seven_days_in_a_row}")
print()
print(f"Number of times an employee works 2 ft shifts in a row: {two_ft_days_in_a_row}")
print()

SUMMARY

Number of times shifts jump forwards 2.5 hours: 5
Number of times shifts jump backwards 1.5 hours: 5

Number of times an employee gets 2 days off after midnights: 2
Number of times an employee gets 3 days off after midnights: 1
Number of times an employee gets 4 days off after midnights: 8
Number of times an employee gets 5 days off after midnights: 1

Number of times an employee works 2 midnights in a row: 2

Number of times there is a day shift within 1 days of a late shift: 8
Number of times there is a day shift within 2 days of a late shift: 2
Number of times there is a day shift within 3 days of a late shift: 1

Number of times there is a afternoon shift within 1 days of a late shift: 8
Number of times there is a afternoon shift within 2 days of a late shift: 2

Number of times an employee works 3 late days in a row: 3
Number of times an employee works 4 late days in a row: 0
Number of times an employee works 5 late days in a row: 0

Number of times an employee works 5 da